In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

nb_classes = 10
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0

train_y = tf.keras.utils.to_categorical(train_y, nb_classes)
test_y = tf.keras.utils.to_categorical(test_y, nb_classes)

TensorFlow 1.x selected.
170500096/170498071 [==============================] - 3s 0us/step


In [2]:
class Block(keras.layers.Layer):
  def __init__(self, f1, f2, s):
    super().__init__()
    kernel_initializer = 'he_normal'
    self.strides = s
    
    self.layers = keras.Sequential([
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
      layers.ReLU(),
    ])
    
  def call(self, inputs):
    x = self.layers(inputs)
    
    return x

class Network(keras.Model):
  def __init__(self):
    super().__init__()
    self.kernel_initializer = 'he_normal'
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=self.kernel_initializer),
      layers.ReLU()
    ])

    self.blocks = []
    filters_per_stage = [16, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [3, 3, 3]
    for filters, strides, repeats in zip(filters_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(Block(filters, filters, strides))
        strides = 1

    self.last_layer = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=self.kernel_initializer)
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    for block in self.blocks:
      x = block(x)
    x = self.last_layer(x)

    return x

model = Network()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(0.005),
              metrics=['accuracy'])

epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 50000 samples, validate on 10000 samples
Epoch 1/7
50000/50000 [==============================] - 20s 391us/sample - loss: 2.3043 - acc: 0.0992 - val_loss: 2.3032 - val_acc: 0.1000
Epoch 2/7
50000/50000 [==============================] - 13s 254us/sample - loss: 2.3030 - acc: 0.0995 - val_loss: 2.3027 - val_acc: 0.1000
Epoch 3/7
50000/50000 [==============================] - 13s 253us/sample - loss: 2.3029 - acc: 0.0990 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 4/7
50000/50000 [==============================] - 13s 254us/sample - loss: 2.3030 - acc: 0.0963 - val_loss: 2.3027 - val_acc: 0.1000
Epoch 5/7
50000/50000 [==============================] - 13s 254us/sample - loss: 2.3029 - acc: 0.1008 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 6/7
50000/50000 [==============================] - 13s 254us/sample - loss: 2.3030 - acc: 0.1007 - val_loss: 2.3027 - val_acc: 0.1000
Epoch 7/7
50000/50000 [======

In [3]:
class Block(keras.layers.Layer):
  def __init__(self, f1, f2, s):
    super().__init__()
    kernel_initializer = 'he_normal'
    self.strides = s
    
    self.layers = keras.Sequential([
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU(),
    ])
    
  def call(self, inputs):
    x = self.layers(inputs)
    
    return x

class Network(keras.Model):
  def __init__(self):
    super().__init__()
    self.kernel_initializer = 'he_normal'
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=self.kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU()
    ])

    self.blocks = []
    filters_per_stage = [16, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [3, 3, 3]
    for filters, strides, repeats in zip(filters_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(Block(filters, filters, strides))
        strides = 1

    self.last_layer = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=self.kernel_initializer)
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    for block in self.blocks:
      x = block(x)
    x = self.last_layer(x)

    return x

model = Network()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(0.005),
              metrics=['accuracy'])

epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))

Train on 50000 samples, validate on 10000 samples
Epoch 1/7
50000/50000 [==============================] - 20s 404us/sample - loss: 1.5250 - acc: 0.4283 - val_loss: 12.6360 - val_acc: 0.2027
Epoch 2/7
50000/50000 [==============================] - 19s 373us/sample - loss: 1.0978 - acc: 0.6014 - val_loss: 2.8554 - val_acc: 0.3872
Epoch 3/7
50000/50000 [==============================] - 19s 375us/sample - loss: 0.8985 - acc: 0.6803 - val_loss: 1.5641 - val_acc: 0.5409
Epoch 4/7
50000/50000 [==============================] - 19s 374us/sample - loss: 0.7662 - acc: 0.7302 - val_loss: 1.2343 - val_acc: 0.6041
Epoch 5/7
50000/50000 [==============================] - 19s 379us/sample - loss: 0.6787 - acc: 0.7640 - val_loss: 1.3119 - val_acc: 0.6285
Epoch 6/7
50000/50000 [==============================] - 19s 375us/sample - loss: 0.6100 - acc: 0.7869 - val_loss: 1.1021 - val_acc: 0.6790
Epoch 7/7
50000/50000 [==============================] - 19s 376us/sample - loss: 0.5494 - acc: 0.8100 - val_

In [ ]:
%%time
class ResNetBlock(keras.layers.Layer):
  def __init__(self, f1, f2, s):
    super().__init__()
    kernel_initializer = 'he_normal'
    self.strides = s
    
    self.residual = keras.Sequential([
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer, use_bias=False),
      layers.BatchNormalization()
    ])
    
    if self.strides == 2:
      self.shortcut_conv = keras.Sequential([
        layers.Conv2D(filters=f2, kernel_size=1, strides=s, kernel_initializer=kernel_initializer, padding='same', use_bias=False),
        layers.BatchNormalization()
      ])

  def call(self, inputs):
    if self.strides == 2:
      shortcut = self.shortcut_conv(inputs)
    else:
      shortcut = inputs
    
    x = self.residual(inputs)
    
    x = keras.layers.add([x, shortcut]) # Or, x = x + shortcut
    x = keras.layers.ReLU()(x) # Or, x = tf.nn.relu(x)
    
    return x

class ResNet(keras.Model):
  def __init__(self):
    super().__init__()
    self.kernel_initializer = 'he_normal'
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=self.kernel_initializer, use_bias=False),
      layers.BatchNormalization(),
      layers.ReLU()
    ])

    self.blocks = []
    filters_per_stage = [16, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [5, 5, 5]
    for filters, strides, repeats in zip(filters_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(ResNetBlock(filters, filters, strides))
        strides = 1

    self.last_layer = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=self.kernel_initializer)
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    for block in self.blocks:
      x = block(x)
    x = self.last_layer(x)

    return x

model = ResNet()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(0.005),
              metrics=['accuracy'])

epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))